In [1]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt

In [2]:
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders
        self.order = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] < self.dataclose[-1]:
                    # current close less than previous close

                    if self.dataclose[-1] < self.dataclose[-2]:
                        # previous close less than the previous close

                        # BUY, BUY, BUY!!! (with default parameters)
                        self.log('BUY CREATE, %.2f' % self.dataclose[0])

                        # Keep track of the created order to avoid a 2nd order
                        self.order = self.buy(size=2)

        else:

            # Already in the market ... we might sell
            if len(self) >= (self.bar_executed + 5):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell(size=1)

In [3]:
import yfinance as yf
import pandas as pd

ticker = "AAPL"
df = yf.download(ticker, start="2019-01-01", end="2024-01-01")

df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
df.columns = ['open', 'high', 'low', 'close', 'volume']

df.index = pd.to_datetime(df.index)

# Convert DataFrame to Backtrader feed
data = bt.feeds.PandasData(dataname=df)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [4]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(TestStrategy)

# Datas are in a subfolder of the samples. Need to find where the script is
# because it could have been called from anywhere

# Create a Data Feed
# data = bt.feeds.YahooFinanceData(
#     dataname='./nasdaq100_stock_prices.csv',
#     # Do not pass values before this date
#     fromdate=datetime.datetime(2000, 1, 1),
#     # Do not pass values before this date
#     todate=datetime.datetime(2000, 12, 31),
#     # Do not pass values after this date
#     reverse=False)

# Add the Data Feed to Cerebro
cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(100000.0)

# Add a FixedSize sizer according to the stake
# cerebro.addsizer(bt.sizers.FixedSize, stake=10)

# Set the commission - 0.1% ... divide by 100 to remove the %
# cerebro.broker.setcommission(commission=0.001)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
cerebro.run()

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2019-01-02, Close, 37.67
2019-01-02, BUY CREATE, 37.67
2019-01-03, BUY EXECUTED, 34.34
2019-01-03, Close, 33.92
2019-01-04, Close, 35.36
2019-01-07, Close, 35.28
2019-01-08, Close, 35.96
2019-01-09, Close, 36.57
2019-01-10, Close, 36.68
2019-01-10, SELL CREATE, 36.68
2019-01-11, SELL EXECUTED, 36.47
2019-01-11, Close, 36.32
2019-01-14, Close, 35.78
2019-01-15, Close, 36.51
2019-01-16, Close, 36.96
2019-01-17, Close, 37.18
2019-01-18, Close, 37.40
2019-01-18, SELL CREATE, 37.40
2019-01-22, SELL EXECUTED, 37.31
2019-01-22, Close, 36.57
2019-01-23, Close, 36.71
2019-01-24, Close, 36.42
2019-01-25, Close, 37.63
2019-01-28, Close, 37.28
2019-01-29, Close, 36.89
2019-01-29, BUY CREATE, 36.89
2019-01-30, BUY EXECUTED, 38.94
2019-01-30, Close, 39.42
2019-01-31, Close, 39.70
2019-02-01, Close, 39.72
2019-02-04, Close, 40.85
2019-02-05, Close, 41.55
2019-02-06, Close, 41.56
2019-02-06, SELL CREATE, 41.56
2019-02-07, SELL EXECUTED, 41.12
2019-02-07, Close, 40.7

In [9]:
cerebro.plot()

<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 4 Axes>]]